In [3]:
import numpy as np
import cv2
import os
import pickle
import imutils
import time
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
from imutils.video import VideoStream

In [5]:
confidence_value = 0.50

net = cv2.dnn.readNetFromCaffe("./deploy.prototxt.txt", "./res10_300x300_ssd_iter_140000.caffemodel")

videos = os.listdir("./videos/")
for video in videos:
	if(video == "processados"):
		break
	infos = video.split(".")[0].split("_")
	nome = infos[0]
	tipo = infos[1]

	vs = cv2.VideoCapture("./videos/"+video)
	saved_path = "./dataset/"+tipo
	read = 0
	saved = 0

	# loop over frames from the video file stream
	while True:
		# grab the frame from the file
		(grabbed, frame) = vs.read()
		# if the frame was not grabbed, then we have reached the end
		# of the stream
		if read >= 100:
			break
		# increment the total number of frames read thus far
		read += 1
		
		# grab the frame dimensions and construct a blob from the frame
		(h, w) = frame.shape[:2]
		blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
			(300, 300), (104.0, 177.0, 123.0))
		# pass the blob through the network and obtain the detections and
		# predictions
		net.setInput(blob)
		detections = net.forward()
		# ensure at least one face was found
		if len(detections) > 0:
			# we're making the assumption that each image has only ONE
			# face, so find the bounding box with the largest probability
			i = np.argmax(detections[0, 0, :, 2])
			confidence = detections[0, 0, i, 2]
			# ensure that the detection with the largest probability also
			# means our minimum probability test (thus helping filter out
			# weak detections)
			if confidence > confidence_value:
				# compute the (x, y)-coordinates of the bounding box for
				# the face and extract the face ROI
				box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
				(startX, startY, endX, endY) = box.astype("int")
				face = frame[startY:endY, startX:endX]
				# write the frame to disk
				p = os.path.sep.join([saved_path,"{}_{}_{}.png".format(nome,tipo,saved)])
				cv2.imwrite(p, face)
				saved += 1
				print("[INFO] saved {} to disk".format(p))
	# do a bit of cleanup
	vs.release()
	cv2.destroyAllWindows()

[INFO] saved ./dataset/falso\bruno_falso_0.png to disk
[INFO] saved ./dataset/falso\bruno_falso_1.png to disk
[INFO] saved ./dataset/falso\bruno_falso_2.png to disk
[INFO] saved ./dataset/falso\bruno_falso_3.png to disk
[INFO] saved ./dataset/falso\bruno_falso_4.png to disk
[INFO] saved ./dataset/falso\bruno_falso_5.png to disk
[INFO] saved ./dataset/falso\bruno_falso_6.png to disk
[INFO] saved ./dataset/falso\bruno_falso_7.png to disk
[INFO] saved ./dataset/falso\bruno_falso_8.png to disk
[INFO] saved ./dataset/falso\bruno_falso_9.png to disk
[INFO] saved ./dataset/falso\bruno_falso_10.png to disk
[INFO] saved ./dataset/falso\bruno_falso_11.png to disk
[INFO] saved ./dataset/falso\bruno_falso_12.png to disk
[INFO] saved ./dataset/falso\bruno_falso_13.png to disk
[INFO] saved ./dataset/falso\bruno_falso_14.png to disk
[INFO] saved ./dataset/falso\bruno_falso_15.png to disk
[INFO] saved ./dataset/falso\bruno_falso_16.png to disk
[INFO] saved ./dataset/falso\bruno_falso_17.png to disk
[I

In [1]:
def build(width, height, depth, classes):
	# initialize the model along with the input shape to be
	# "channels last" and the channels dimension itself
	model = Sequential()
	inputShape = (height, width, depth)
	chanDim = -1
	# if we are using "channels first", update the input shape
	# and channels dimension
	if K.image_data_format() == "channels_first":
		inputShape = (depth, height, width)
		chanDim = 1
		
	# first CONV => RELU => CONV => RELU => POOL layer set
	model.add(Conv2D(16, (3, 3), padding="same",
		input_shape=inputShape))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(16, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	# second CONV => RELU => CONV => RELU => POOL layer set
	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))

	# first (and only) set of FC => RELU layers
	model.add(Flatten())
	model.add(Dense(64))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	# softmax classifier
	model.add(Dense(classes))
	model.add(Activation("softmax"))
	# return the constructed network architecture
	return model

In [4]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
BS = 8
EPOCHS = 50
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images("./dataset/"))
data = []
labels = []
# loop over all image paths
for imagePath in imagePaths:
	# extract the class label from the filename, load the image and
	# resize it to be a fixed 32x32 pixels, ignoring aspect ratio
	label = imagePath.split(os.path.sep)[-2]
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (32, 32))
	# update the data and labels lists, respectively
	data.append(image)
	labels.append("Falso" if label == "./dataset/falso" else "Real")
# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")
# initialize the optimizer and model
print("[INFO] compiling model...")
model = build(width=32, height=32, depth=3, classes=len(le.classes_))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS), validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS, epochs=EPOCHS)

[INFO] loading images...
[INFO] compiling model...



[INFO] training network for 50 epochs...
Epoch 1/50


75/75 [==============================] - 4s 17ms/step - loss: 0.3034 - accuracy: 0.8717 - val_loss: 1.1419 - val_accuracy: 0.5100
Epoch 2/50
75/75 [==============================] - 1s 13ms/step - loss: 0.1852 - accuracy: 0.9283 - val_loss: 2.3166 - val_accuracy: 0.5100
Epoch 3/50
75/75 [==============================] - 1s 15ms/step - loss: 0.1491 - accuracy: 0.9500 - val_loss: 1.9061 - val_accuracy: 0.5100
Epoch 4/50
75/75 [==============================] - 1s 13ms/step - loss: 0.1306 - accuracy: 0.9617 - val_loss: 2.3881 - val_accuracy: 0.5100
Epoch 5/50
75/75 [==============================] - 1s 16ms/step - loss: 0.1571 - accuracy: 0.9567 - val_loss: 0.7259 - val_accuracy: 0.6700
Epoch 6/50
75/75 [==============================] - 1s 13ms/step - loss: 0.1049 - accuracy: 0.9567 - val_loss: 1.6653 - val_accuracy: 0.5350
Epoch 7/50
75/75 [==============================] - 1s 14

KeyboardInterrupt: 

In [6]:
model_teste = "liveness.keras"

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=le.classes_))
# save the network to disk
print("[INFO] serializing network to '{}'...".format(model_teste))
model.save(model_teste)
# save the label encoder to disk
f = open("le.pickle", "wb")
f.write(pickle.dumps(le))
f.close()
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

[INFO] evaluating network...
61/61 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

       Falso       1.00      1.00      1.00       259
        Real       1.00      1.00      1.00       229

    accuracy                           1.00       488
   macro avg       1.00      1.00      1.00       488
weighted avg       1.00      1.00      1.00       488

[INFO] serializing network to 'liveness.keras'...


In [7]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
net = cv2.dnn.readNetFromCaffe("./deploy.prototxt.txt", "./res10_300x300_ssd_iter_140000.caffemodel")
# load the liveness detector model and label encoder from disk
print("[INFO] loading liveness detector...")
model = load_model("liveness.keras")
le = pickle.loads(open("le.pickle", "rb").read())
# initialize the video stream and allow the camera sensor to warmup
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 600 pixels
	frame = vs.read()
	frame = imutils.resize(frame, width=600)
	# grab the frame dimensions and convert it to a blob
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
		(300, 300), (104.0, 177.0, 123.0))
	# pass the blob through the network and obtain the detections and
	# predictions
	net.setInput(blob)
	detections = net.forward()
		# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with the
		# prediction
		confidence = detections[0, 0, i, 2]
		# filter out weak detections
		if confidence > 0.50:
			# compute the (x, y)-coordinates of the bounding box for
			# the face and extract the face ROI
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")
			# ensure the detected bounding box does fall outside the
			# dimensions of the frame
			startX = max(0, startX)
			startY = max(0, startY)
			endX = min(w, endX)
			endY = min(h, endY)
			# extract the face ROI and then preproces it in the exact
			# same manner as our training data
			face = frame[startY:endY, startX:endX]
			face = cv2.resize(face, (32, 32))
			face = face.astype("float") / 255.0
			face = img_to_array(face)
			face = np.expand_dims(face, axis=0)
			# pass the face ROI through the trained liveness detector
			# model to determine if the face is "real" or "fake"
			preds = model.predict(face)[0]
			j = np.argmax(preds)
			label = le.classes_[j]
			# draw the label and bounding box on the frame
			if(label == "Real"):
				label = "{}: {:.4f}".format(label, preds[j])
				cv2.putText(frame, label, (startX, startY - 10),
					cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
				cv2.rectangle(frame, (startX, startY), (endX, endY),
					(255, 0, 0), 2)
			else:
				label = "{}: {:.4f}".format(label, preds[j])
				cv2.putText(frame, label, (startX, startY - 10),
					cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
				cv2.rectangle(frame, (startX, startY), (endX, endY),
					(0, 0, 255), 2)
				# show the output frame and wait for a key press
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] loading face detector...
[INFO] loading liveness detector...
[INFO] starting video stream...
1/1 [==============================] - 0s 26ms/step
